### Preprocessing the Data for a Neural Network

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Trial 1: Drop columns with low correlation with Label

In [3]:
application_df.corr()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
EIN,1.000000,0.004942,0.030241,-0.044482
STATUS,0.004942,1.000000,0.000383,-0.001636
ASK_AMT,0.030241,0.000383,1.000000,-0.010861
IS_SUCCESSFUL,-0.044482,-0.001636,-0.010861,1.000000


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' as well as STATUS since the correlation between status and IS_SUCCESSFUL is low
application_df = application_df.drop(columns = ['EIN', 'NAME', 'STATUS', 'ASK_AMT'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,1


In [5]:
# Determine which values to replace if counts are less than ...?
values = application_df.APPLICATION_TYPE.value_counts()
replace_application = values[values < 500].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ..?
values = application_df.CLASSIFICATION.value_counts()
replace_class =  values[values < 1000].index.tolist()
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
data_types = application_df.dtypes
application_cat = data_types[data_types == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(application_cat, 1)
application_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(['IS_SUCCESSFUL'], 1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=X.shape[1], activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3360      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.5706 - accuracy: 0.7202
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7275
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7300
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7309
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5492 - accuracy: 0.7300
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7325
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7308
Epoch 8/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7318
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5453 - accuracy: 0.7328
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5445 - accuracy: 0.7345

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.5875 - accuracy: 0.6597 - 414ms/epoch - 2ms/step
Loss: 1.5875102281570435, Accuracy: 0.6597084403038025


### Trial 2: Add hidden Layer

In [20]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=X.shape[1], activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3360      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 25)                775       
                                                                 
 dense_9 (Dense)             (None, 15)                390       
                                                                 
 dense_10 (Dense)            (None, 1)                 16        
                                                                 
Total params: 6,971
Trainable params: 6,971
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5726 - accuracy: 0.7185
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7291
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7302
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7321
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7297
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7325
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7321
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7343
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7341
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7346

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.3986 - accuracy: 0.7195 - 431ms/epoch - 2ms/step
Loss: 1.39860200881958, Accuracy: 0.7195335030555725


### Trial 3: Change units per hidden layers

In [24]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, input_dim=X.shape[1], activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=70, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 100)               4200      
                                                                 
 dense_12 (Dense)            (None, 70)                7070      
                                                                 
 dense_13 (Dense)            (None, 40)                2840      
                                                                 
 dense_14 (Dense)            (None, 10)                410       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 14,531
Trainable params: 14,531
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5700 - accuracy: 0.7196
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7296
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7297
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7332
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7309
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7330
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7345
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7343
Epoch 9/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5442 - accuracy: 0.7336
Epoch 10/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5435 - accuracy: 0.7354

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 2.4197 - accuracy: 0.6596 - 853ms/epoch - 3ms/step
Loss: 2.4197158813476562, Accuracy: 0.6595918536186218


### Trial 4: Change activation of output layer

In [31]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, input_dim=X.shape[1], activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=70, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 100)               4200      
                                                                 
 dense_22 (Dense)            (None, 70)                7070      
                                                                 
 dense_23 (Dense)            (None, 40)                2840      
                                                                 
 dense_24 (Dense)            (None, 10)                410       
                                                                 
 dense_25 (Dense)            (None, 1)                 11        
                                                                 
Total params: 14,531
Trainable params: 14,531
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5728 - accuracy: 0.7176
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7280
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7298
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7299
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7312
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7320
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7325
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7330
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7338
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7346

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 2.4842 - accuracy: 0.6292 - 453ms/epoch - 2ms/step
Loss: 2.4842453002929688, Accuracy: 0.6291545033454895


## Trial 5: Change activation of hidden layers

In [35]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=150, input_dim=X.shape[1], activation="sigmoid"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 150)               6300      
                                                                 
 dense_27 (Dense)            (None, 100)               15100     
                                                                 
 dense_28 (Dense)            (None, 50)                5050      
                                                                 
 dense_29 (Dense)            (None, 10)                510       
                                                                 
 dense_30 (Dense)            (None, 1)                 11        
                                                                 
Total params: 26,971
Trainable params: 26,971
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.6187 - accuracy: 0.6723
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5784 - accuracy: 0.7278
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5771 - accuracy: 0.7275
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5766 - accuracy: 0.7276
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5756 - accuracy: 0.7305
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5757 - accuracy: 0.7303
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5747 - accuracy: 0.7304
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5739 - accuracy: 0.7285
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5707 - accuracy: 0.7287
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5679 - accuracy: 0.7291

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6083 - accuracy: 0.6905 - 438ms/epoch - 2ms/step
Loss: 0.6082682609558105, Accuracy: 0.6904956102371216


### Trial 6: Increase number of epochs

In [41]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=X.shape[1], activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 80)                3360      
                                                                 
 dense_37 (Dense)            (None, 30)                2430      
                                                                 
 dense_38 (Dense)            (None, 25)                775       
                                                                 
 dense_39 (Dense)            (None, 15)                390       
                                                                 
 dense_40 (Dense)            (None, 1)                 16        
                                                                 
Total params: 6,971
Trainable params: 6,971
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5715 - accuracy: 0.7178
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7267
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7293
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7301
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7302
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7329
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7322
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7333
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accuracy: 0.7342
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7348

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 3.9277 - accuracy: 0.6505 - 422ms/epoch - 2ms/step
Loss: 3.9277231693267822, Accuracy: 0.6504956483840942


### Trial 7: Saving model every 5 epochs for best combination of params

In [53]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=X.shape[1], activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 80)                3360      
                                                                 
 dense_52 (Dense)            (None, 30)                2430      
                                                                 
 dense_53 (Dense)            (None, 25)                775       
                                                                 
 dense_54 (Dense)            (None, 15)                390       
                                                                 
 dense_55 (Dense)            (None, 1)                 16        
                                                                 
Total params: 6,971
Trainable params: 6,971
Non-trainable params: 0
_________________________________________________________________


In [58]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint
freq = 804 * 5
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Create a Callback that saves the weights every 5 epochs
os.makedirs("models/",exist_ok=True)
checkpoint_path = "models/model.{epoch:02d}.hdf5"
cp= ModelCheckpoint(filepath=checkpoint_path, save_freq=freq)

In [59]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=20, callbacks = cp)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5396 - accuracy: 0.7369
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5387 - accuracy: 0.7379
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5390 - accuracy: 0.7370
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5390 - accuracy: 0.7377
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5383 - accuracy: 0.7378
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5379 - accuracy: 0.7382
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5381 - accuracy: 0.7371
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5384 - accuracy: 0.7385
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5376 - accuracy: 0.7384
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5377 - accuracy: 0.7386

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 2.5214 - accuracy: 0.6718 - 432ms/epoch - 2ms/step
Loss: 2.521430730819702, Accuracy: 0.6718367338180542


In [61]:
nn.save("AlphabetSoupCharity_Optimized.h5")